In [35]:
import pandas as pd
import numpy as np
import pprint as pp

# Functions

In [ ]:
def trova_voti_lista_plurinom(df, lista):
    
    sub = df[df['LISTA'] == lista]
    sub = sub[['COLLEGIOPLURINOMINALE', 
               'VOTI_LISTA']].groupby('COLLEGIOPLURINOMINALE').sum().reset_index()
    
    sub.columns = ['COLLEGIOPLURINOMINALE', 'VOTI_LISTA_{}'.format(lista)]
    
    return sub


def get_metadata_plurinom_camera(path):
    
    meta = pd.read_excel(path, sheet_name = 'Table ')
    
    # Fix columns 
    meta.columns = meta.iloc[0].tolist()
    
    # Drop first row
    meta = meta.drop(axis = 0, index = 0)

    # Fill NA for CIRCOSCRIZIONI
    meta['CIRCOSCRIZIONI'] = meta['CIRCOSCRIZIONI'].fillna(method = 'ffill')
    
    # Remove last columns who is an NA
    meta = meta.iloc[:-1, :-1]

    # Further remove NAs
    meta = meta.dropna()
    
    return meta

def get_metadata_plurinom_senato(path):
    
    meta = pd.read_excel(path, sheet_name = 'Table 9')
    
    # Fix columns 
    meta.columns = meta.iloc[0].tolist()
    
    # Drop first row
    meta = meta.drop(axis = 0, index = 0)

    # Fill NA for CIRCOSCRIZIONI
    meta['CIRCOSCRIZIONI'] = meta['CIRCOSCRIZIONI'].fillna(method = 'ffill')
    
    # Remove last columns who is an NA
    meta = meta.iloc[:-4, :-1]
    
    # Fix seggi plurinominali
    meta['Seggi plurinominali'] = meta['Seggi plurinominali'].replace('-', np.nan)

    # Further remove NAs
    meta = meta.dropna()
    
    return meta

# Analisi Camera

In [274]:
# Read Data
camera = pd.read_csv('camera-20180304.txt', sep = ";", encoding = "ISO-8859-1")

In [275]:
# Crea tabella elettori e votanti collegio plurinominale
plurinom = camera[['COLLEGIOPLURINOMINALE',
                   'COLLEGIOUNINOMINALE',
                   'COMUNE', 
                   'ELETTORI',
                   'VOTANTI']].drop_duplicates()

plurinom = plurinom.groupby('COLLEGIOPLURINOMINALE').agg({'ELETTORI': np.sum,
                                               'VOTANTI': np.sum
                                              })

plurinom.columns = ['ELETTORI_PLURINOMINALE', 'VOTANTI_PLURINOMINALE']

plurinom = plurinom.reset_index()

#.agg({'ELETTORI': np.sum(),'VOTANTI': np.sum()})

In [276]:
meta_camera.rename(columns={"Collegi plurinominali":"COLLEGIOPLURINOMINALE"}, inplace = True)

In [277]:
# Aggiungi metatadati collegi plurinominali

path = 'commissione-collegi-elettorali_relazione_13-novembre-2020-con-errata-corrige.xlsx'
meta_camera = get_metadata_plurinom_camera(path)
meta_camera.rename(columns={"Collegi plurinominali":"COLLEGIOPLURINOMINALE"}, inplace = True)
meta_camera['COLLEGIOPLURINOMINALE'] = [x.upper() for x in meta_camera['COLLEGIOPLURINOMINALE']]

plurinom = pd.merge(plurinom, meta_camera, on = "COLLEGIOPLURINOMINALE", how = 'left')

In [278]:
# Trova percentuali di lista

sub_pap = trova_voti_lista_plurinom(camera, 'POTERE AL POPOLO!')
sub_sin_riv = trova_voti_lista_plurinom(camera, 'PER UNA SINISTRA RIVOLUZIONARIA')
sub_part_com = trova_voti_lista_plurinom(camera, 'PARTITO COMUNISTA')
sub_part_lib = trova_voti_lista_plurinom(camera, 'LIBERI E UGUALI')
sub_part_m5s = trova_voti_lista_plurinom(camera, 'MOVIMENTO 5 STELLE')

In [279]:
rep = pd.merge(plurinom, sub_pap, on = 'COLLEGIOPLURINOMINALE', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_sin_riv, on = 'COLLEGIOPLURINOMINALE', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_com, on = 'COLLEGIOPLURINOMINALE', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_m5s, on = 'COLLEGIOPLURINOMINALE', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_lib, on = 'COLLEGIOPLURINOMINALE', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep_camera = rep.sort_values('VOTI_LISTA_POTERE AL POPOLO!', ascending = False).fillna(0)

In [280]:
rep_camera

,COLLEGIOPLURINOMINALE,ELETTORI_PLURINOMINALE,VOTANTI_PLURINOMINALE,CIRCOSCRIZIONI,Popolazione residente,Seggi totali,Popolazione\nmedia per seggio,Seggi uninominali,Popolazione\nmedia\nper seggio uninominale,Seggi plurinominali,...,VOTI_LISTA_POTERE AL POPOLO!,VOTI_LISTA_POTERE AL POPOLO!_PERC,VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA,VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC,VOTI_LISTA_PARTITO COMUNISTA,VOTI_LISTA_PARTITO COMUNISTA_PERC,VOTI_LISTA_MOVIMENTO 5 STELLE,VOTI_LISTA_MOVIMENTO 5 STELLE_PERC,VOTI_LISTA_LIBERI E UGUALI,VOTI_LISTA_LIBERI E UGUALI_PERC
54,TOSCANA - 03,755718,600864,Toscana,1280561,8,160070,3,426854,5,...,13720.0,2.283379,1054.0,0.175414,5526.0,0.919676,124592.0,20.735474,32942.0,5.482439
16,LAZIO 1 - 01,1005501,720702,Lazio 1,1163634,8,145454,3,387878,5,...,13583.0,1.884690,595.0,0.082558,3610.0,0.500901,190302.0,26.405088,29446.0,4.085739
17,LAZIO 1 - 02,991777,712410,Lazio 1,1258261,8,157282,3,419420,5,...,12784.0,1.794472,627.0,0.088011,3688.0,0.517679,210817.0,29.592089,31272.0,4.389607
6,CAMPANIA 1 - 02,746750,451924,Campania 1,1629108,11,148100,4,407277,7,...,12227.0,2.705543,568.0,0.125685,0.0,0.000000,223455.0,49.445261,13637.0,3.017543
53,TOSCANA - 02,606754,473354,Toscana,1205308,8,150663,3,401769,5,...,10804.0,2.282436,834.0,0.176189,5263.0,1.111853,116081.0,24.523084,22006.0,4.648952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,LOMBARDIA 2 - 01,686975,520034,Lombardia 2,871886,6,145314,2,435943,4,...,2552.0,0.490737,1074.0,0.206525,0.0,0.000000,110266.0,21.203614,10967.0,2.108901
51,SICILIA 2 - 03,787593,492382,Sicilia 2,877027,6,146171,2,438514,4,...,2460.0,0.499612,0.0,0.000000,1334.0,0.270928,246040.0,49.969333,10986.0,2.231194
49,SICILIA 2 - 01,645421,420631,Sicilia 2,823275,5,164655,2,411638,3,...,2407.0,0.572236,0.0,0.000000,1187.0,0.282195,173950.0,41.354536,11604.0,2.758713
47,SICILIA 1 - 02,709562,446059,Sicilia 1,1121878,7,160268,3,373959,4,...,2174.0,0.487379,0.0,0.000000,969.0,0.217236,196510.0,44.054710,10338.0,2.317631


# Analisi Senato

In [281]:
# Read Data
senato = pd.read_csv('senato-20180304.txt', sep = ";", encoding = "ISO-8859-1")

In [282]:
# Crea tabella elettori e votanti collegio plurinominale
plurinom = camera[['COLLEGIOPLURINOMINALE',
                   'COLLEGIOUNINOMINALE',
                   'COMUNE', 
                   'ELETTORI',
                   'VOTANTI']].drop_duplicates()

plurinom = plurinom.groupby('COLLEGIOPLURINOMINALE').agg({'ELETTORI': np.sum,
                                               'VOTANTI': np.sum
                                              })

plurinom.columns = ['ELETTORI_PLURINOMINALE', 'VOTANTI_PLURINOMINALE']# Crea tabella elettori e votanti collegio plurinominale
plurinom = camera[['COLLEGIOPLURINOMINALE',
                   'COLLEGIOUNINOMINALE',
                   'COMUNE', 
                   'ELETTORI',
                   'VOTANTI']].drop_duplicates()

plurinom = plurinom.groupby('COLLEGIOPLURINOMINALE').agg({'ELETTORI': np.sum,
                                               'VOTANTI': np.sum
                                              })

plurinom.columns = ['ELETTORI_PLURINOMINALE', 'VOTANTI_PLURINOMINALE']

In [283]:
# Aggiungi metatadati collegi plurinominali

path = 'commissione-collegi-elettorali_relazione_13-novembre-2020-con-errata-corrige.xlsx'
meta_senato = get_metadata_plurinom_camera(path)
meta_senato.rename(columns={"Collegi plurinominali":"COLLEGIOPLURINOMINALE"}, inplace = True)
meta_senato['COLLEGIOPLURINOMINALE'] = [x.upper() for x in meta_camera['COLLEGIOPLURINOMINALE']]

plurinom = pd.merge(plurinom, meta_senato, on = "COLLEGIOPLURINOMINALE", how = 'left')

In [284]:
sub_pap = trova_voti_lista_plurinom(camera, 'POTERE AL POPOLO!')
sub_sin_riv = trova_voti_lista_plurinom(camera, 'PER UNA SINISTRA RIVOLUZIONARIA')
sub_part_com = trova_voti_lista_plurinom(camera, 'PARTITO COMUNISTA')
sub_part_lib = trova_voti_lista_plurinom(camera, 'LIBERI E UGUALI')
sub_part_m5s = trova_voti_lista_plurinom(camera, 'MOVIMENTO 5 STELLE')

In [285]:
rep = pd.merge(plurinom, sub_pap, on = 'COLLEGIOPLURINOMINALE', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_sin_riv, on = 'COLLEGIOPLURINOMINALE', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_com, on = 'COLLEGIOPLURINOMINALE', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_m5s, on = 'COLLEGIOPLURINOMINALE', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_lib, on = 'COLLEGIOPLURINOMINALE', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep_senato = rep.sort_values('VOTI_LISTA_POTERE AL POPOLO!_PERC', ascending = False).fillna(0)

# Scenario Analysis



In [286]:
# Scenario 1 - PAP si allea con forze comuniste che si assume mantengano stessi risultati del 2018. 
# Anche PAP mantiene gli stessi risultati del 2018.

rep_camera['Scenario 1'] = (rep_camera['VOTI_LISTA_POTERE AL POPOLO!_PERC'] + 
                            rep_camera['VOTI_LISTA_PARTITO COMUNISTA_PERC'] +
                            rep_camera['VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC'])

rep_senato['Scenario 1'] = (rep_senato['VOTI_LISTA_POTERE AL POPOLO!_PERC'] + 
                            rep_senato['VOTI_LISTA_PARTITO COMUNISTA_PERC'] +
                            rep_senato['VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC'])

rep_camera.sort_values(by = 'Scenario 1', ascending = False)

,COLLEGIOPLURINOMINALE,ELETTORI_PLURINOMINALE,VOTANTI_PLURINOMINALE,CIRCOSCRIZIONI,Popolazione residente,Seggi totali,Popolazione\nmedia per seggio,Seggi uninominali,Popolazione\nmedia\nper seggio uninominale,Seggi plurinominali,...,VOTI_LISTA_POTERE AL POPOLO!_PERC,VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA,VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC,VOTI_LISTA_PARTITO COMUNISTA,VOTI_LISTA_PARTITO COMUNISTA_PERC,VOTI_LISTA_MOVIMENTO 5 STELLE,VOTI_LISTA_MOVIMENTO 5 STELLE_PERC,VOTI_LISTA_LIBERI E UGUALI,VOTI_LISTA_LIBERI E UGUALI_PERC,Scenario 1
53,TOSCANA - 02,606754,473354,Toscana,1205308,8,150663,3,401769,5,...,2.282436,834.0,0.176189,5263.0,1.111853,116081.0,24.523084,22006.0,4.648952,3.570478
54,TOSCANA - 03,755718,600864,Toscana,1280561,8,160070,3,426854,5,...,2.283379,1054.0,0.175414,5526.0,0.919676,124592.0,20.735474,32942.0,5.482439,3.378468
6,CAMPANIA 1 - 02,746750,451924,Campania 1,1629108,11,148100,4,407277,7,...,2.705543,568.0,0.125685,0.0,0.000000,223455.0,49.445261,13637.0,3.017543,2.831228
55,TOSCANA - 04,628951,485973,0,0,0,0,0,0,0,...,1.289784,1020.0,0.209888,5063.0,1.041827,111197.0,22.881312,15095.0,3.106140,2.541499
16,LAZIO 1 - 01,1005501,720702,Lazio 1,1163634,8,145454,3,387878,5,...,1.884690,595.0,0.082558,3610.0,0.500901,190302.0,26.405088,29446.0,4.085739,2.468149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,SICILIA 1 - 03,637968,383260,0,0,0,0,0,0,0,...,0.667954,0.0,0.000000,0.0,0.000000,182365.0,47.582581,9243.0,2.411679,0.667954
61,VENETO 2 - 02,652441,524221,Veneto 2,859205,6,143200,2,429603,4,...,0.597839,0.0,0.000000,0.0,0.000000,111885.0,21.343098,10835.0,2.066876,0.597839
59,VENETO 1 - 02,720398,560123,0,0,0,0,0,0,0,...,0.589335,0.0,0.000000,0.0,0.000000,112942.0,20.163785,12136.0,2.166667,0.589335
60,VENETO 2 - 01,712555,573175,Veneto 2,1163710,7,166244,3,387903,4,...,0.573996,0.0,0.000000,0.0,0.000000,128636.0,22.442709,15062.0,2.627819,0.573996


In [287]:
# Scenario 2 - PAP corre da sola e riesce a conquistare il 10% dei voti di liberi & uguali e il 5% dei 
# voti del movimento cinque stelle

rep_camera['Scenario 2'] = (rep_camera['VOTI_LISTA_POTERE AL POPOLO!_PERC'] + 
                            rep_camera['VOTI_LISTA_MOVIMENTO 5 STELLE_PERC'] * 0.05 +
                            rep_camera['VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC'] * 0.1
                           )

rep_senato['Scenario 2'] = (rep_senato['VOTI_LISTA_POTERE AL POPOLO!_PERC'] + 
                            rep_senato['VOTI_LISTA_MOVIMENTO 5 STELLE_PERC'] * 0.05 +
                            rep_senato['VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC'] * 0.1
                           )

rep_camera.sort_values(by = 'Scenario 2', ascending = False)

,COLLEGIOPLURINOMINALE,ELETTORI_PLURINOMINALE,VOTANTI_PLURINOMINALE,CIRCOSCRIZIONI,Popolazione residente,Seggi totali,Popolazione\nmedia per seggio,Seggi uninominali,Popolazione\nmedia\nper seggio uninominale,Seggi plurinominali,...,VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA,VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC,VOTI_LISTA_PARTITO COMUNISTA,VOTI_LISTA_PARTITO COMUNISTA_PERC,VOTI_LISTA_MOVIMENTO 5 STELLE,VOTI_LISTA_MOVIMENTO 5 STELLE_PERC,VOTI_LISTA_LIBERI E UGUALI,VOTI_LISTA_LIBERI E UGUALI_PERC,Scenario 1,Scenario 2
6,CAMPANIA 1 - 02,746750,451924,Campania 1,1629108,11,148100,4,407277,7,...,568.0,0.125685,0.0,0.000000,223455.0,49.445261,13637.0,3.017543,2.831228,5.190375
5,CAMPANIA 1 - 01,918321,610227,Campania 1,1425848,9,158427,3,475283,6,...,725.0,0.118808,0.0,0.000000,334073.0,54.745693,17584.0,2.881551,1.446839,4.077196
7,CAMPANIA 1 - 03,735932,506642,0,0,0,0,0,0,0,...,632.0,0.124743,0.0,0.000000,244459.0,48.250836,12644.0,2.495648,1.228086,3.528359
53,TOSCANA - 02,606754,473354,Toscana,1205308,8,150663,3,401769,5,...,834.0,0.176189,5263.0,1.111853,116081.0,24.523084,22006.0,4.648952,3.570478,3.526209
8,CAMPANIA 2 - 01,582865,425263,Campania 2,1189821,8,148727,3,396607,5,...,0.0,0.000000,1392.0,0.327327,171497.0,40.327280,10351.0,2.434023,1.686721,3.375758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,VENETO 2 - 02,652441,524221,Veneto 2,859205,6,143200,2,429603,4,...,0.0,0.000000,0.0,0.000000,111885.0,21.343098,10835.0,2.066876,0.597839,1.664994
30,LOMBARDIA 3 - 02,838085,671572,Lombardia 3,1201656,8,150207,3,400552,5,...,0.0,0.000000,0.0,0.000000,107979.0,16.078544,13802.0,2.055178,0.843096,1.647024
59,VENETO 1 - 02,720398,560123,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.000000,112942.0,20.163785,12136.0,2.166667,0.589335,1.597524
27,LOMBARDIA 2 - 01,686975,520034,Lombardia 2,871886,6,145314,2,435943,4,...,1074.0,0.206525,0.0,0.000000,110266.0,21.203614,10967.0,2.108901,0.697262,1.571570


# Export to Excel

In [288]:


with pd.ExcelWriter("plurinominali.xlsx") as writer:
    rep_camera.sort_values(by = 'COLLEGIOPLURINOMINALE').to_excel(writer, sheet_name="Plurinominali Camera", index = False)  
    rep_senato.sort_values(by = 'COLLEGIOPLURINOMINALE').to_excel(writer, sheet_name="Plurinominali Senato", index = False)  
    
    